# Simple Search Engine
Elasticsearch와 mysql을 결합한 검색엔진을 만들어보겠습니다.
____

In [1]:
import pandas as pd
df = pd.read_csv("movie_doc.csv",sep="\t")

In [2]:
df.head()

,id,title,story,gender,link,image
0,0,인비저블 게스트,"의문의 습격으로 살해 당한 ‘로라’ ‘아드리안’은 연인의 죽음에 절망하고, 범인은 ...",N,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20170828_179/1...
1,1,"나, 다니엘 블레이크",평생을 성실하게 목수로 살아가던 다니엘은 지병인 심장병이 악화되어 일을 계속 해나갈...,F,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20161117_246/1...
2,2,국가부도의 날,"1997년, 대한민국 최고의 경제 호황을 믿어 의심치 않았던 그때, 곧 엄청난 경제...",M,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20181105_232/1...
3,3,당갈,전직 레슬링 선수였던 ‘마하비르 싱 포갓(아미르 칸)’은 아버지의 반대로 금메달의 ...,F,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20180329_2/152...
4,4,스파이더맨: 파 프롬 홈,"‘엔드게임’ 이후 변화된 세상, 스파이더맨 ‘피터 파커’는 학교 친구들과 유럽 여행...",M,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20190527_181/1...


## 2. MySQL에 데이터 적재

In [3]:
import mysql.connector


HOST = "localhost"
USER = "root"
PASSWORD = "123456789"
mydb = mysql.connector.connect(
  host=HOST,
  user=USER,
  password=PASSWORD,
  database="movie_db"
)

mycursor = mydb.cursor()


In [5]:
sql = "SELECT * FROM movie"
mycursor = mydb.cursor(dictionary=True)
mycursor.execute(sql)


myresult = mycursor.fetchall()
for x in myresult:
    print(x)
    break

{'id': '0', 'story': '의문의 습격으로 살해 당한 ‘로라’ ‘아드리안’은 연인의 죽음에 절망하고, 범인은 흔적도 없이 사라졌다. 유력한 용의자로 누명을 쓴 ‘아드리안’은 승률 100%의 변호사 ‘버지니아’를 선임한다. 그리고 자신의 무죄를 입증하기 위해 고군분투하던 중 과거 그와 ‘로라’가 은폐한 교통사고와 숨겨진 연관성을 찾게 되는데… 남은 시간은 단 3시간, 사건을 재구성해 무죄를 입증해야 한다!', 'title': '인비저블 게스트', 'link': 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=159516', 'image': 'https://movie-phinf.pstatic.net/20170828_179/1503887362732snsIK_JPEG/movie_image.jpg?type=f67'}


## 3. ElasticSearch에 데이터 인덱싱

In [6]:
from elasticsearch import Elasticsearch, helpers


In [7]:
import pprint  
INDEX_NAME = "movie_index"


INDEX_SETTINGS = {
  "settings" : {
    "index":{
      "analysis":{
        "analyzer":{
          "korean":{
            "type":"custom",
            "tokenizer":"nori_tokenizer"
          }
        }
      }
    }
  },
  "mappings": {

      "properties" : {
        "story" : {
          "type" : "text",
          "analyzer": "korean"
        },
        "title" : {
          "type" : "text",
          "analyzer": "korean"
        }
      }

  }
}


In [8]:


try:
    es.transport.close()
except:
    pass
es = Elasticsearch()



In [9]:
if es.indices.exists(INDEX_NAME):
    es.indices.delete(index=INDEX_NAME)
es.indices.create(index=INDEX_NAME, body=INDEX_SETTINGS)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'movie_index'}

In [10]:
movies = [
    {
        "_index": INDEX_NAME,
        "_id" : doc['id'],
        "_source": {
            "title": doc['title'],
            "story": doc['story'],
        }
    }
    for doc in myresult
]

movies[:10]

[{'_index': 'movie_index',
  '_id': '0',
  '_source': {'title': '인비저블 게스트',
   'story': '의문의 습격으로 살해 당한 ‘로라’ ‘아드리안’은 연인의 죽음에 절망하고, 범인은 흔적도 없이 사라졌다. 유력한 용의자로 누명을 쓴 ‘아드리안’은 승률 100%의 변호사 ‘버지니아’를 선임한다. 그리고 자신의 무죄를 입증하기 위해 고군분투하던 중 과거 그와 ‘로라’가 은폐한 교통사고와 숨겨진 연관성을 찾게 되는데… 남은 시간은 단 3시간, 사건을 재구성해 무죄를 입증해야 한다!'}},
 {'_index': 'movie_index',
  '_id': '1',
  '_source': {'title': '나, 다니엘 블레이크',
   'story': '평생을 성실하게 목수로 살아가던 다니엘은 지병인 심장병이 악화되어 일을 계속 해나갈 수 없는 상황이 된다. 다니엘은 실업급여를 받기 위해 찾아간 관공서에서 복잡하고 관료적인 절차 때문에 번번히 좌절한다. 그러던 어느 날 다니엘은 두 아이와 함께 런던에서 이주한 싱글맘 케이티를 만나 도움을 주게되고, 서로를 의지하게 되는데...'}},
 {'_index': 'movie_index',
  '_id': '2',
  '_source': {'title': '국가부도의 날',
   'story': '1997년, 대한민국 최고의 경제 호황을 믿어 의심치 않았던 그때, 곧 엄청난 경제 위기가 닥칠 것을 예견한 한국은행 통화정책팀장 ‘한시현’(김혜수)은 이 사실을 보고하고, 정부는 뒤늦게 국가부도 사태를 막기 위한 비공개 대책팀을 꾸린다. 한편, 곳곳에서 감지되는 위기의 시그널을 포착하고 과감히 사표를 던진 금융맨 ‘윤정학’(유아인)은 국가부도의 위기에 투자하는 역베팅을 결심, 투자자들을 모으기 시작한다. 이런 상황을 알 리 없는 작은 공장의 사장이자 평범한 가장 ‘갑수’(허준호)는 대형 백화점과의 어음 거래 계약서에 도장을 찍고 소박한 행복을 꿈꾼다. 국가부도까지 남은 시간 

In [11]:
try:
    response = helpers.bulk(es, movies)
    print ("\nRESPONSE:", response)
except Exception as e:
    print("\nERROR:", e)


RESPONSE: (403, [])


In [12]:
doc = es.get(index=INDEX_NAME, id=1)
pprint.pprint(doc)

{'_id': '1',
 '_index': 'movie_index',
 '_primary_term': 1,
 '_seq_no': 1,
 '_source': {'story': '평생을 성실하게 목수로 살아가던 다니엘은 지병인 심장병이 악화되어 일을 계속 해나갈 수 없는 상황이 '
                      '된다. 다니엘은 실업급여를 받기 위해 찾아간 관공서에서 복잡하고 관료적인 절차 때문에 번번히 '
                      '좌절한다. 그러던 어느 날 다니엘은 두 아이와 함께 런던에서 이주한 싱글맘 케이티를 만나 도움을 '
                      '주게되고, 서로를 의지하게 되는데...',
             'title': '나, 다니엘 블레이크'},
 '_type': '_doc',
 '_version': 1,
 'found': True}


## 4. ElasticSearch에서 검색후, 검색결과를 MySQL에서 재 검색

In [13]:
query="은퇴한 킬러"
res = es.search(index=INDEX_NAME, q=query)
for hit in res['hits']['hits']:
    print("Doc ID: %3r  Score: %5.2f" % (hit['_id'], hit['_score']))
    print("Title: {}".format(hit['_source']['title']))
    print("줄거리: {}".format(hit['_source']['story']))


Doc ID: '67'  Score: 11.31
Title: 존 윅 - 리로드
줄거리: 업계 최고의 레전드 킬러 ‘존 윅’은 과거를 뒤로한 채 은퇴를 선언하지만, 과거 자신의 목숨을 구했던 옛 동료와 피로 맺은 암살자들의 룰에 의해 로마로 향한다. ‘국제 암살자 연합’을 탈취하려는 옛 동료의 계획으로 ‘존 윅’은 함정에 빠지게 되고, 전세계 암살자들의 총구는 그를 향하는데...
Doc ID: '54'  Score:  6.70
Title: 헌터 킬러
줄거리: 미 국방부는 격침당한 잠수함의 행방을 찾기 위해 ‘헌터 킬러’를 극비리에 투입시키고 캡틴 ‘글래스’(제라드 버틀러)는 배후에 숨겨진 음모가 있음을 알게 된다. 한편, 지상에서는 VIP가 납치되어 전세계는 초긴장 상태에 놓이게 되는데… 일촉즉발 위기상황, VIP를 구출하라! 단 한 척의 공격 잠수함 ‘헌터 킬러’와 최정예 특수부대 네이비 씰의 숨막히는 육해공 합동 작전이 펼쳐진다!
Doc ID: '165'  Score:  6.28
Title: 안나
줄거리: 파리의 톱모델로 위장한 강력한 킬러 ‘안나’(사샤 루스)가 살아남기 위해 모든 위협을 제거해나가는 하드코어 킬링 액션
Doc ID: '42'  Score:  6.06
Title: 레옹
줄거리: 한 손엔 우유 2팩이 든 가방, 다른 한 손엔 화분을 들고 뿌리 없이 떠도는 킬러 레옹은 어느 날 옆집 소녀 마틸다의 일가족이 몰살 당하는 것을 목격한다. 그 사이 심부름을 갔다 돌아 온 마틸다는 가족들이 처참히 몰살 당하자 레옹에게 도움을 청한다. 가족의 원수를 갚기 위해 킬러가 되기로 결심한 12세 소녀 마틸다는 레옹에게 글을 알려주는 대신 복수하는 법을 배우게 된다. 드디어 그녀는 가족을 죽인 사람이 부패 마약 경찰 스탠스임을 알게 되고, 그의 숙소로 향하게 되는데…
Doc ID: '181'  Score:  5.31
Title: 수상한 그녀
줄거리: 아들 자랑이 유일한 낙인 욕쟁이 칠순 할매 오말순(나문희分)은 어느 날, 가족들이 자신을 요양원으로 독립(?)시키

In [14]:
movie_id = res['hits']['hits'][0]["_id"]

In [15]:
sql = "SELECT * FROM movie WHERE id = {}".format(movie_id)
mycursor = mydb.cursor(dictionary=True)
mycursor.execute(sql)


myresult = mycursor.fetchall()
for x in myresult:
    print(x)

{'id': '67', 'story': '업계 최고의 레전드 킬러 ‘존 윅’은 과거를 뒤로한 채 은퇴를 선언하지만, 과거 자신의 목숨을 구했던 옛 동료와 피로 맺은 암살자들의 룰에 의해 로마로 향한다. ‘국제 암살자 연합’을 탈취하려는 옛 동료의 계획으로 ‘존 윅’은 함정에 빠지게 되고, 전세계 암살자들의 총구는 그를 향하는데...', 'title': '존 윅 - 리로드', 'link': 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=143932', 'image': 'https://movie-phinf.pstatic.net/20200807_193/1596789703698oLgpX_JPEG/movie_image.jpg?type=f67'}


In [ ]:
#쿼리---> 엘라스틱서치 --> MySQL